In [7]:
import sqlite3
import pandas as pd


In [8]:
conn = sqlite3.connect('MusicPlayer.db') 

In [9]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Interactions
                               ''', conn)

In [10]:
df = pd.DataFrame(sql_query, columns = ['id', 'user_id', 'song_id' , 'like' , 'listen_count'])

In [11]:
print (df)


        id  user_id  song_id  like  listen_count
0        1        1      159     1           791
1        2        1       95     1           260
2        3        1      124     1           973
3        4        1       97     1           341
4        5        1      173     1           897
...    ...      ...      ...   ...           ...
3118  3119      156      112     1           947
3119  3120      156      480     1           655
3120  3121      133      439     0             3
3121  3122      133      242     0             1
3122  3123      133      184     0             1

[3123 rows x 5 columns]


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import time 
import joblib
import app
from app import Interactions, db
from sqlalchemy import create_engine
import model.Recommenders as Recommenders
import model.Evaluation as Evaluation


In [15]:
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM Songs
                               ''', conn)

In [16]:
song_df = pd.DataFrame(sql_query, columns = ['id', 'name', 'path' , 'artist' , 'genre' , 'cover_photo' , 'duration' , 'total_likes' , 'total_listen_count'])

In [17]:
print(song_df)

      id                                     name  \
0      1                      Bikhoti Ka Mela.mp3   
1      2                      Chakdait Chhora.mp3   
2      3                         Jawa Pardesh.mp3   
3      4                     Kan Full Khilana.mp3   
4      5                   Latuli Teri Mukhdi.mp3   
..   ...                                      ...   
557  558  Hum Jane Pahchane Hain Kanihya Tere.mp3   
558  559   Rang De Chunirya Shyam Naam Ki (1).mp3   
559  560       Rang De Chunirya Shyam Naam Ki.mp3   
560  561        Shyam Dhani Ke Male Main Jana.mp3   
561  562        Jai Shiv Shankar Jai Mahadeva.mp3   

                                                  path         artist  \
0          Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3           Anil   
1          Dataset\garhwali\Anil - Chakdait Chhora.mp3           Anil   
2             Dataset\garhwali\Anil - Jawa Pardesh.mp3           Anil   
3         Dataset\garhwali\Anil - Kan Full Khilana.mp3           Anil  

In [18]:
song_df.head()

,id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,5,426
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,7,582
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,9,27
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,5,564


In [19]:
song_df.rename(columns = {'id':'song_id'}, inplace = True)
song_df.head()

,song_id,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1,2,Chakdait Chhora.mp3,Dataset\garhwali\Anil - Chakdait Chhora.mp3,Anil,garhwali,/static/images/garhwali.jpeg,5:39,5,426
2,3,Jawa Pardesh.mp3,Dataset\garhwali\Anil - Jawa Pardesh.mp3,Anil,garhwali,/static/images/garhwali.jpeg,6:19,7,582
3,4,Kan Full Khilana.mp3,Dataset\garhwali\Anil - Kan Full Khilana.mp3,Anil,garhwali,/static/images/garhwali.jpeg,9:44,9,27
4,5,Latuli Teri Mukhdi.mp3,Dataset\garhwali\Anil - Latuli Teri Mukhdi.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:56,5,564


In [20]:
merged_song_df = pd.merge(df, song_df.drop_duplicates(['song_id']), on="song_id", how="left")

In [21]:
merged_song_df.head()

,id,user_id,song_id,like,listen_count,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
0,1,1,159,1,791,Yun Sajaa Chand.mp3,Dataset\ghazal\Pankaj Udhas - Yun Sajaa Chand.mp3,Pankaj Udhas,ghazal,/static/images/ghazal.jpeg,4:53,12,40
1,2,1,95,1,260,Yeh Dil Pagal Dil Mera.mp3,Dataset\ghazal\Ghulam Ali - Yeh Dil Pagal Dil ...,Ghulam Ali,ghazal,/static/images/ghazal.jpeg,8:6,11,403
2,3,1,124,1,973,Shayad Main Zindagi Ki Sahar.mp3,Dataset\ghazal\Jagjit Singh - Shayad Main Zind...,Jagjit Singh,ghazal,/static/images/ghazal.jpeg,6:23,13,753
3,4,1,97,1,341,Haath Chute Bhi.mp3,Dataset\ghazal\Gulzar - Haath Chute Bhi.mp3,Gulzar,ghazal,/static/images/ghazal.jpeg,5:27,2,431
4,5,1,173,1,897,Khubsurat Hai Aankhen.mp3,Dataset\ghazal\Talat Aziz - Khubsurat Hai Aank...,Talat Aziz,ghazal,/static/images/ghazal.jpeg,6:39,4,897


In [22]:
merged_song_df.sort_values(by='song_id')

,id,user_id,song_id,like,listen_count,name,path,artist,genre,cover_photo,duration,total_likes,total_listen_count
1823,1824,92,1,1,715,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
3019,3020,151,1,1,420,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
839,840,42,1,1,441,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
1004,1005,51,1,1,93,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
741,742,38,1,1,600,Bikhoti Ka Mela.mp3,Dataset\garhwali\Anil - Bikhoti Ka Mela.mp3,Anil,garhwali,/static/images/garhwali.jpeg,7:6,6,420
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1770,1771,89,562,1,240,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
2723,2724,137,562,1,12,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
499,500,25,562,1,93,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983
2966,2967,149,562,1,78,Jai Shiv Shankar Jai Mahadeva.mp3,Dataset\bhajan\Ravindra Jain - Jai Shiv Shanka...,Ravindra Jain,bhajan,/static/images/bhajan.jpeg,18:24,13,983


In [23]:
len(merged_song_df)

3123

In [24]:
#Creating a subset of our dataset

In [25]:
ong_df =ong_df = merged_song_df.head(100)

In [26]:
print(ong_df)

     id  user_id  song_id  like  listen_count  \
0     1        1      159     1           791   
1     2        1       95     1           260   
2     3        1      124     1           973   
3     4        1       97     1           341   
4     5        1      173     1           897   
..  ...      ...      ...   ...           ...   
95   96        5       63     1           250   
96   97        5      222     1           702   
97   98        5      163     1           538   
98   99        5       81     1           961   
99  100        5      204     1           505   

                                name  \
0                Yun Sajaa Chand.mp3   
1         Yeh Dil Pagal Dil Mera.mp3   
2   Shayad Main Zindagi Ki Sahar.mp3   
3                Haath Chute Bhi.mp3   
4          Khubsurat Hai Aankhen.mp3   
..                               ...   
95        Mor Moriki Maand Pelya.mp3   
96      Moh Moh Ke Dhaage (Male).mp3   
97                 Ae jane Wafaa.mp3   
98         

In [27]:
song_grouped = merged_song_df.groupby([‘song’]).agg({‘listen_count’: ‘count’}).reset_index()

SyntaxError: invalid character in identifier (<ipython-input-27-14c7224fddc3>, line 1)

In [28]:
train_data, test_data = train_test_split(merged_song_df, test_size = 0.20, random_state=0)

In [29]:
print(train_data.head(5))

        id  user_id  song_id  like  listen_count  \
502    503       26       11     1            55   
600    601       31      532     1           838   
398    399       20      496     1           230   
1836  1837       92       38     1           853   
2900  2901      146      153     1           196   

                                        name  \
502                      Mera Baju Ranga.mp3   
600                    Sukh Ke Sab Sathi.mp3   
398                     Piyawa Se Pahile.mp3   
1836                 Chakhuli Meri Fur E.mp3   
2900  Jeeye to Jeeye Kaise (From Saajan).mp3   

                                                   path               artist  \
502    Dataset\garhwali\Anil Bist - Mera Baju Ranga.mp3            Anil Bist   
600   Dataset\bhajan\Bipin Sachdeva - Sukh Ke Sab Sa...       Bipin Sachdeva   
398   Dataset\bhojpuri\Ritesh Pandey - Piyawa Se Pah...        Ritesh Pandey   
1836  Dataset\garhwali\Narender Singh Negi - Chakhul...  Narender Singh Negi  

In [30]:
pm = Recommenders.popularity_recommender_py()

In [31]:
pm.create(train_data, 'user_id', 'name')

In [32]:
user_id = 1

In [33]:
pm.recommend(user_id)

,user_id,name,score,Rank
312,1,Mera Aapki Kripa Se.mp3,15,1.0
84,1,Chandi Jaisa Rang,14,2.0
343,1,Nawazish Karam Shukriya Meharbani,13,3.0
403,1,Rang De Chunirya Shyam Naam Ki.mp3,13,4.0
431,1,Sancha Naam Tera,13,5.0
460,1,Sriman Narayana.mp3,13,6.0
72,1,Bol Pinjare Ka Tota Ram.mp3,12,7.0
188,1,Hridaya Banalo Bhakto.mp3,12,8.0
246,1,Kanha Teri Murli Ki (1).mp3,12,9.0
374,1,Payoji Maine Ram Ratan Dhan.mp3,12,10.0


In [34]:
pm.recommend(9)

,user_id,name,score,Rank
312,9,Mera Aapki Kripa Se.mp3,15,1.0
84,9,Chandi Jaisa Rang,14,2.0
343,9,Nawazish Karam Shukriya Meharbani,13,3.0
403,9,Rang De Chunirya Shyam Naam Ki.mp3,13,4.0
431,9,Sancha Naam Tera,13,5.0
460,9,Sriman Narayana.mp3,13,6.0
72,9,Bol Pinjare Ka Tota Ram.mp3,12,7.0
188,9,Hridaya Banalo Bhakto.mp3,12,8.0
246,9,Kanha Teri Murli Ki (1).mp3,12,9.0
374,9,Payoji Maine Ram Ratan Dhan.mp3,12,10.0


In [35]:
is_model = Recommenders.item_similarity_recommender_py()

In [36]:
is_model.create(train_data, 'user_id', 'name')

In [37]:
user_id=7

In [38]:
user_items = is_model.get_user_items(user_id)

In [39]:
for user_item in user_items:
    print(user_item)

Tikuli Satale Bani.mp3
Saaiyaan.mp3
Allahu Akbar.mp3
Sanu Ek Pal Chain.mp3
Tum Se Hi.mp3
Khamoshi
Jeena Jeena (From Badlapur).mp3
Thand Mathu Chaumaas.mp3
Lagake Facewash.mp3
Tere Sang Yaara.mp3
Aaja Re Moray Saiyaan
Sarkela Seeena Se Odhaniya.mp3
Bheegi Bheegi.mp3
Daal De Kewadi Mein Killi.mp3
Nai-Lagda-_From-Notebook_.mp3
Outro.mp3
Kal Ho Naa Ho.mp3


In [40]:
is_model.recommend(user_id)

No. of unique songs for the user: 17
no. of unique songs in the training set: 546
Non zero values in cooccurence_matrix :1349


,user_id,song,score,rank
0,7,Channa Mereya.mp3,0.065403,1
1,7,Mehrama (From Love Aaj Kal).mp3,0.047857,2
2,7,Mehrama.mp3,0.042037,3
3,7,Uddi Ja (Coke Studio Season 9).mp3,0.041963,4
4,7,Zehnaseeb.mp3,0.041226,5
5,7,Chithhi Aayi Hai.mp3,0.039988,6
6,7,Faqeer.mp3,0.039757,7
7,7,Piya Milenge.mp3,0.039714,8
8,7,Kabhi Yun Bhi To Ho.mp3,0.039542,9
9,7,Ramchandra Kah Gaye.mp3,0.039453,10


In [41]:
is_model.get_similar_items(['Saiyaan'])

no. of unique songs in the training set: 546
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,Baith Basanti Baith Meri.mp3,0.0,1
1,,Chakdait Chhora.mp3,0.0,2
2,,Na Suno.mp3,0.0,3
3,,Roots (feat Raja Kumari).mp3,0.0,4
4,,Dil.mp3,0.0,5
5,,Badfella.mp3,0.0,6
6,,Nat Roop Ch Na Rang.mp3,0.0,7
7,,Humsafar.mp3,0.0,8
8,,Maugi Ke Bada Nakhra.mp3,0.0,9
9,,Haath Chute Bhi.mp3,0.0,10


In [42]:
is_model.get_similar_items(['Kun Faya Kun'])

no. of unique songs in the training set: 546
Non zero values in cooccurence_matrix :0


,user_id,song,score,rank
0,,Baith Basanti Baith Meri.mp3,0.0,1
1,,Chakdait Chhora.mp3,0.0,2
2,,Na Suno.mp3,0.0,3
3,,Roots (feat Raja Kumari).mp3,0.0,4
4,,Dil.mp3,0.0,5
5,,Badfella.mp3,0.0,6
6,,Nat Roop Ch Na Rang.mp3,0.0,7
7,,Humsafar.mp3,0.0,8
8,,Maugi Ke Bada Nakhra.mp3,0.0,9
9,,Haath Chute Bhi.mp3,0.0,10


In [43]:
is_model.get_similar_items(['Mere Gully Mein (feat Naezy).mp3'])

no. of unique songs in the training set: 546
Non zero values in cooccurence_matrix :49


,user_id,song,score,rank
0,,Banda Ban Ja.mp3,0.333333,1
1,,Kya Baat Ay.mp3,0.333333,2
2,,Khuda Jaane.mp3,0.333333,3
3,,Nat Roop Ch Na Rang.mp3,0.250000,4
4,,Runjhun Barkha Lagi Chaumasi.mp3,0.250000,5
5,,Khojyali Min Khojyali.mp3,0.250000,6
6,,Do Botalaan.mp3,0.200000,7
7,,Koi Chaahat Koi Hasrat.mp3,0.200000,8
8,,Hamar Saiya Aaj Ladan Se.mp3,0.200000,9
9,,Dawood.mp3,0.200000,10


## Quantitative comparison between the models¶
We now formally compare the popularity and the personalized models using precision-recall curves.

Class to calculate precision and recall (This can be used as a black box)

In [44]:
#Evaluation.precision_recall_calculator

# Matrix Factorization based Recommender System

Using SVD matrix factorization based collaborative filtering recommender system -------------------------------------------------------------------------------- The following code implements a Singular Value Decomposition (SVD) based matrix factorization collaborative filtering recommender system. The user ratings matrix used is a small matrix as follows: Item0 Item1 Item2 Item3 User0 3 1 2 3 User1 4 3 4 3 User2 3 2 1 5 User3 1 6 5 2 User4 0 0 5 0 As we can see in the above matrix, all users except user 4 rate all items. The code calculates predicted recommendations for user 4

In [45]:

#Code source written with help from: 
#http://antoinevastel.github.io/machine%20learning/python/2016/02/14/svd-recommender-system.html

import math as mt
import csv
from sparsesvd import sparsesvd #used for matrix factorization
import numpy as np
from scipy.sparse import csc_matrix #used for sparse matrix
from scipy.sparse.linalg import * #used for matrix multiplication

#Note: You may need to install the library sparsesvd. Documentation for 
#sparsesvd method can be found here:
#https://pypi.python.org/pypi/sparsesvd/

In [46]:
#constants defining the dimensions of our User Rating Matrix (URM)
MAX_PID = 4
MAX_UID = 5

#Compute SVD of the user ratings matrix
def computeSVD(urm, K):
    U, s, Vt = sparsesvd(urm, K)

    dim = (len(s), len(s))
    S = np.zeros(dim, dtype=np.float32)
    for i in range(0, len(s)):
        S[i,i] = mt.sqrt(s[i])

    U = csc_matrix(np.transpose(U), dtype=np.float32)
    S = csc_matrix(S, dtype=np.float32)
    Vt = csc_matrix(Vt, dtype=np.float32)
    
    return U, S, Vt

#Compute estimated rating for the test user
def computeEstimatedRatings(urm, U, S, Vt, uTest, K, test):
    rightTerm = S*Vt 

    estimatedRatings = np.zeros(shape=(MAX_UID, MAX_PID), dtype=np.float16)
    for userTest in uTest:
        prod = U[userTest, :]*rightTerm
        #we convert the vector to dense format in order to get the indices 
        #of the movies with the best estimated ratings 
        estimatedRatings[userTest, :] = prod.todense()
        recom = (-estimatedRatings[userTest, :]).argsort()[:250]
    return recom


In [47]:
#Used in SVD calculation (number of latent factors)
K=2

#Initialize a sample user rating matrix
urm = np.array([[3, 1, 2, 3],[4, 3, 4, 3],[3, 2, 1, 5], [1, 6, 5, 2], [5, 0,0 , 0]])
urm = csc_matrix(urm, dtype=np.float32)

#Compute SVD of the input user ratings matrix
U, S, Vt = computeSVD(urm, K)

#Test user set as user_id 4 with ratings [0, 0, 5, 0]
uTest = [4]
print("User id for whom recommendations are needed: %d" % uTest[0])

#Get estimated rating for test user
print("Predictied ratings:")
uTest_recommended_items = computeEstimatedRatings(urm, U, S, Vt, uTest, K, True)
print(uTest_recommended_items)

User id for whom recommendations are needed: 4
Predictied ratings:
[0 3 2 1]
